# Model to forecast inventory demand based on historical sales data. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import time
import random
import pickle
import math
import warnings
warnings.filterwarnings("ignore")

## Model accuracy is RMSLE

In [2]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

## Load Training Data 
The size of the training data is quite large (~4 GB). Large datasets require significant amount of memory to process. Instead, we will sample the data randomly for our initial data analysis and visualization. 

In [3]:
def load_samp_data(filename='train.csv', columns=[], load_pkl=1):
    """ 
      Function returns a dataframe containing the training data sampled randomly. 
      The data is also stored in a pickle file for later processing.
    """
    if load_pkl:
        inputfile = open('train_samp_data.pkl', 'rb')
        data = pickle.load(inputfile)
        inputfile.close()
        return data
    
    chunksize= 10 ** 6
    datasize = 74180464 #datasize = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
    samplesize = 10 ** 3 # samples per chunk of data read from the file.
    
    data = pd.DataFrame([],columns=columns)
    chunks = pd.read_csv(filename, iterator=True, chunksize=chunksize)
    for chunk in chunks:
        chunk.columns = columns
        data = data.append(chunk.sample(samplesize)) 
    
    # write data to a pickle file.
    outputfile = open('train_samp_data.pkl','wb')
    pickle.dump(data,outputfile)
    outputfile.close()
    
    return data
 
load_pkl = 0
columns = ['week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client_id', 'prod_id', 'saleunit_curr_wk', 'saleamt_curr_wk', 'retunit_next_week', 'retamt_next_wk', 'y_pred_demand']
tic = time.time()
train_data_samp = load_samp_data('train.csv', columns, load_pkl)
toc = time.time()
print '*********'
print 'Time to load: ', toc-tic, 'sec'
print 
print train_data_samp.describe()
print '*********'
print train_data_samp[['week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client_id', 'prod_id']]

features_train = train_data_samp[['week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client_id', 'prod_id']].values
labels_train_sale = train_data_samp[['saleunit_curr_wk']].values
labels_train_return = train_data_samp[['retunit_next_week']].values
labels_train = train_data_samp[['y_pred_demand']].values

*********
Time to load:  75.1288080215 sec

           week_num  sales_depot_id  sales_chan_id      route_id  \
count  75000.000000    75000.000000   75000.000000  75000.000000   
mean       5.982800     2761.836040       1.378893   2114.386307   
std        2.027004     4603.625646       1.455427   1492.045185   
min        3.000000     1110.000000       1.000000      1.000000   
25%        4.000000     1312.000000       1.000000   1161.000000   
50%        6.000000     1614.000000       1.000000   1283.000000   
75%        8.000000     2040.000000       1.000000   2802.000000   
max        9.000000    25759.000000      11.000000   9935.000000   

          client_id       prod_id  saleunit_curr_wk  saleamt_curr_wk  \
count  7.500000e+04  75000.000000      75000.000000     75000.000000   
mean   1.798237e+06  20879.108587          7.275120        67.941156   
std    1.832623e+06  18659.089843         19.898198       258.362483   
min    1.050000e+02     72.000000          0.000000    

## Feature Engineering 


In [33]:

train_data_samp.groupby(['client_id', 'prod_id']).sum()

,,week_num,sales_depot_id,sales_chan_id,route_id,saleunit_curr_wk,saleamt_curr_wk,retunit_next_week,retamt_next_wk,y_pred_demand
client_id,prod_id,,,,,,,,,
105.0,43065.0,4.0,2061.0,2.0,7222.0,33.0,163.68,0.0,0.0,33.0
791.0,35651.0,9.0,1227.0,1.0,1263.0,1.0,7.50,0.0,0.0,1.0
1311.0,31717.0,5.0,2095.0,11.0,3911.0,12.0,72.00,0.0,0.0,12.0
2003.0,43285.0,9.0,2012.0,1.0,2006.0,8.0,42.24,0.0,0.0,8.0
2025.0,1242.0,4.0,2012.0,1.0,1159.0,5.0,38.20,0.0,0.0,5.0
2056.0,37058.0,9.0,2012.0,1.0,2804.0,1.0,7.50,0.0,0.0,1.0
2059.0,1232.0,5.0,2012.0,1.0,1157.0,6.0,109.44,0.0,0.0,6.0
2073.0,43066.0,5.0,2012.0,1.0,2006.0,1.0,9.63,0.0,0.0,1.0
2098.0,1242.0,5.0,2012.0,1.0,1159.0,2.0,15.28,0.0,0.0,2.0


### Predict sale units $y_{sale}$ and returns $y_{return}$ using two different classifiers. We will use xgboost to fit $y_{sale}$ and  $y_{return}$ with the input data.

In [ ]:
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from operator import itemgetter

clf = RandomForestClassifier(n_estimators=10)

# specify parameters and distributions to sample from
param_dist = {"max_depth": [10],
              "max_features": sp_randint(4, 7),
              }

# run randomized search
n_iter_search = 10
random_search_sale = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, n_jobs=4, cv=5)
start = time.time()
random_search_sale.fit(features_train, np.ravel(labels_train_sale))
predict = random_search_sale.predict(features_train)

print 'Model Report ********'
print 'Accuracy : ', rmsle(np.ravel(labels_train_sale), predict)
print 'Model Report ********'

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search_sale.grid_scores_)
print random_search_sale.best_score_ 
print random_search_sale.best_estimator_ 
feat_imp = pd.Series(random_search_sale.best_estimator_.feature_importances_).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from operator import itemgetter

clf = RandomForestClassifier(n_estimators=15)

# specify parameters and distributions to sample from
param_dist = {"max_depth": [10],
              "max_features": sp_randint(3, 5),
              }

# run randomized search
n_iter_search = 10
random_search_return = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, n_jobs=4, cv=5)
start = time.time()
random_search_return.fit(features_train, np.ravel(labels_train_return))
predict = random_search_return.predict(features_train)

print 'Model Report ********'
print 'Accuracy : ', rmsle(np.ravel(labels_train_return), predict)
print 'Model Report ********'

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search_return.grid_scores_)
print random_search_return.best_score_ 
print random_search_return.best_estimator_ 
feat_imp = pd.Series(random_search_return.best_estimator_.feature_importances_).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')

In [ ]:
predict_sale = random_search_sale.predict(features_train)
predict_return = random_search_return.predict(features_train)
y_pred = [max(0,(predict_sale[i]-predict_return[i])) for i in xrange(len(predict_return))]
plt.scatter(y_pred,np.ravel(labels_train))
print 'Model Report ********'
print 'Accuracy : ', rmsle(y_pred, np.ravel(labels_train))
print 'Model Report ********'


### 3. Gradient Boosting

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4
from sklearn import metrics

In [ ]:
def modelfit(alg, Xtrain, ytrain, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(Xtrain, label=ytrain)
        print alg.get_params()['n_estimators']
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round = alg.get_params()['n_estimators'], early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        alg.fit(Xtrain, ytrain, eval_metric='auc')
        predict = alg.predict(Xtrain)
        return predict

## Step 1 Fix learning rate and number of estimators for tuning tree-based parameters

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.05,
 n_estimators=100,
 max_depth=15,
 min_child_weight=4,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 scale_pos_weight=1,
 seed=27)

predict = modelfit(xgb1, features_train, np.ravel(labels_train))

In [ ]:
#print model report:
print '\nModel Report ********'
print "Accuracy : %.4g" % rmsle(np.ravel(labels_train), predict)
print '\nModel Report ********'
feat_imp = pd.Series(xgb1.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

## Step 2: Tune max_depth and min_child_weight

In [ ]:
from sklearn.grid_search import GridSearchCV
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=100, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, scale_pos_weight=1, seed=27),  param_grid = param_test1, scoring='roc_auc', n_jobs=4,iid=False)
gsearch1.fit(features_train,np.ravel(labels_train))
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

## Data Cleaning
There are duplicate client ids in cliente_table, which means one client id may have multiple client name that are very similar. We will cluster them based on a hash function and use a clustering algorithm to evaluate similarity.  

In [ ]:
import re
def hash_eval(s):
    hash_base = 4
    s = re.sub('[., ]', '', s)
    seqlen = len(s)
    n = seqlen - 1
    h = 0
    for c in s:
        h += ord(c) * (hash_base ** n)
        n -= 1
    curhash = h
    return curhash

# In the client table, same clients are assigned different client ID. We create a new client table where clients are assigned unique ID. 
clientid_hash = dict()
new_client_id = [-1]   
for idx, s in enumerate(clientnameid_data.NombreCliente):
    t = hash_eval(s)
    clientid_hash.setdefault(t, []).append(clientnameid_data.Cliente_ID[idx])
    if t in clientid_hash:
        a = clientid_hash[t]
        new_client_id.append(a[0])

# In the agency table, same agencies (town, state) are assigned different agency ID. We create a new agency table where agencies (town, state) are assigned unique ID. 
agencyid_hash = dict()
new_agency_id = [-1]   
for idx, s in enumerate(townstate_data.Town+townstate_data.State):
    t = hash_eval(s)
    agencyid_hash.setdefault(t, []).append(townstate_data.Agencia_ID[idx])
    if t in agencyid_hash:
        a = agencyid_hash[t]
        new_agency_id.append(a[0])


In [ ]:
clientnameid_data['New_Cliente_ID'] = new_client_id[1:]
townstate_data['New_Agencia_ID'] = new_agency_id[1:]

In [ ]:
print clientnameid_data.head(10)
print '---'
print townstate_data.head()
print '---'
print train_data_samp.head(10)


In [ ]:
print train_data_samp.head(10)
print '------'
for idx, cid in enumerate(train_data_samp.client_id):
    train_data_samp.client_id.values[idx] = clientnameid_data.New_Cliente_ID[train_data_samp.client_id.values[idx] == clientnameid_data.Cliente_ID.values].values[0]
    train_data_samp.sales_depot_id.values[idx] = townstate_data.New_Agencia_ID[train_data_samp.sales_depot_id.values[idx] == townstate_data.Agencia_ID.values].values[0]
print '-----'
print train_data_samp.head()


## Load Test Data

In [ ]:
test_data = pd.read_csv('test.csv')
test_data.columns = ['id', 'week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client id', 'prod_id']
test_labels = pd.read_csv('sample_submission.csv')
test_data = test_data.drop('id', 1)
print test_data.head()

In [ ]:
g = sns.PairGrid(data_t)
g.map(plt.scatter)

In [ ]:
a = [[1, 2, 3, 4]]

In [ ]:
print a

In [ ]:
np.array(a)

In [ ]:
print np.array(a)

In [ ]:
a = np.array(a)

In [ ]:
a = sp_randint(10,2)

In [ ]:
range(3,10,2)

In [ ]:
sp_randint(1, 6)

In [ ]:
import subprocess 
subprocess.call(['ec2kill'])

In [ ]:
from subprocess import call
call(["ec2-terminate-instances", "i-308b33ed "])

In [ ]:
# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from operator import itemgetter

clf = RandomForestClassifier(n_estimators=30)

# specify parameters and distributions to sample from
param_dist = {"max_depth": [10, None],
              "max_features": sp_randint(1, 6),
              "min_samples_split": sp_randint(1, 6),
              "min_samples_leaf": sp_randint(1, 6),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, n_jobs=4, cv=3)
start = time.time()
random_search.fit(features_train, np.ravel(labels_train))

print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start), n_iter_search))
report(random_search.grid_scores_)
print random_search.best_score_ 
print random_search.best_estimator_ 